In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('jfk20-21data.txt')
winter2 = pd.read_csv ('jfk21-22data.txt')
winter3 = pd.read_csv ('jfk22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
217,JFK,2021-12-09 01:44:00,0.76,-SNRA BR
218,JFK,2021-12-09 01:51:00,1.02,-SN BR
219,JFK,2021-12-09 02:32:00,0.76,-SNRA
220,JFK,2021-12-09 02:51:00,0.76,-SNRA
492,JFK,2021-12-19 11:10:00,T,-PLRA
...,...,...,...,...
2398,JFK,2022-02-25 06:51:00,1.52,-FZRAPL
2399,JFK,2022-02-25 07:51:00,3.05,PL BR
2400,JFK,2022-02-25 08:25:00,2.54,FZRAPL BR
2401,JFK,2022-02-25 08:51:00,3.81,FZRAPL BR


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 51]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 51]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 51]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
jfk = LocationInfo(
    'New York' 'New York', 
    latitude= 40.6446,
    longitude= -73.7797, 
    timezone='America/New_York'
)

jfk

LocationInfo(name='New YorkNew York', region='England', timezone='America/New_York', latitude=40.6446, longitude=-73.7797)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(jfk.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(jfk.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(jfk.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:00:22  2020-12-01 16:28:11  2020-12-01
1   2020-12-02 07:01:22  2020-12-02 16:27:58  2020-12-02
2   2020-12-03 07:02:21  2020-12-03 16:27:48  2020-12-03
3   2020-12-04 07:03:19  2020-12-04 16:27:40  2020-12-04
4   2020-12-05 07:04:16  2020-12-05 16:27:34  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:36:17  2021-02-24 17:40:48  2021-02-24
86  2021-02-25 06:34:49  2021-02-25 17:41:57  2021-02-25
87  2021-02-26 06:33:19  2021-02-26 17:43:06  2021-02-26
88  2021-02-27 06:31:50  2021-02-27 17:44:15  2021-02-27
89  2021-02-28 06:30:19  2021-02-28 17:45:23  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:00:07  2021-12-01 16:28:14  2021-12-01
1   2021-12-02 07:01:07  2021-12-02 16:28:01  2021-12-02
2   2021-12-03 07:02:07  2021-12-03 16:27:50  2021-12-03
3   2021-12-04 07:03:05  2021-12-04 16:27:41  2021-12-04
4   2021-12-05 07:04:02  2021-12-05 16:27:34  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:36:38  2022-02-24 17:40:31  2022-02-24
86  2022-02-25 06:35:10  2022-02-25 17:41:41  2022-02-25
87  2022-02-26 06:33:41  2022-02-26 17:42:50  2022-02-26
88  2022-02-27 06:32:11  2022-02-27 17:43:58  2022-02-27
89  2022-02-28 06:30:41  2022-02-28 17:45:07  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:59:52  2022-12-01 16:28:18  2022-12-01
1   2022-12-02 07:00:53  2022-12-02 16:28:04  2022-12-02
2   2022-12-03 07:01:52  2022-12-03 16:27:52  2022-12-03
3   2022-12-04 07:02:51  2022-12-04 16:27:43  2022-12-04
4   2022-12-05 07:03:48  2022-12-05 16:27:36  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:36:59  2023-02-24 17:40:14  2023-02-24
86  2023-02-25 06:35:31  2023-02-25 17:41:24  2023-02-25
87  2023-02-26 06:34:03  2023-02-26 17:42:33  2023-02-26
88  2023-02-27 06:32:33  2023-02-27 17:43:42  2023-02-27
89  2023-02-28 06:31:03  2023-02-28 17:44:50  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JFK,2020-12-08 11:51:00,0.0000,-SN,2020-12-08,2020-12-08 07:06:58,2020-12-08 16:27:29
1,JFK,2020-12-08 12:51:00,0.0000,-SN,2020-12-08,2020-12-08 07:06:58,2020-12-08 16:27:29
2,JFK,2020-12-09 17:51:00,0.0000,-SN,2020-12-09,2020-12-09 07:07:50,2020-12-09 16:27:33
3,JFK,2020-12-09 18:51:00,0.0000,-SN BR,2020-12-09,2020-12-09 07:07:50,2020-12-09 16:27:33
4,JFK,2020-12-16 19:51:00,0.0001,-SN,2020-12-16,2020-12-16 07:13:05,2020-12-16 16:28:59
...,...,...,...,...,...,...,...
128,JFK,2021-02-20 06:51:00,0.0001,-SN,2021-02-20,2021-02-20 06:42:01,2021-02-20 17:36:08
129,JFK,2021-02-20 07:51:00,0.0001,-SN,2021-02-20,2021-02-20 06:42:01,2021-02-20 17:36:08
130,JFK,2021-02-20 08:51:00,0.0001,-SN,2021-02-20,2021-02-20 06:42:01,2021-02-20 17:36:08
131,JFK,2021-02-22 16:51:00,1.0200,-RASN BR,2021-02-22,2021-02-22 06:39:11,2021-02-22 17:38:29


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JFK,2021-12-09 01:51:00,1.0200,-SN BR,2021-12-09,2021-12-09 07:07:37,2021-12-09 16:27:31
1,JFK,2021-12-09 02:51:00,0.7600,-SNRA,2021-12-09,2021-12-09 07:07:37,2021-12-09 16:27:31
2,JFK,2021-12-24 08:51:00,0.2500,-SN BR,2021-12-24,2021-12-24 07:17:11,2021-12-24 16:32:38
3,JFK,2022-01-07 06:51:00,0.0001,-SN,2022-01-07,2022-01-07 07:19:12,2022-01-07 16:43:57
4,JFK,2022-01-07 07:51:00,1.5200,-SN BR,2022-01-07,2022-01-07 07:19:12,2022-01-07 16:43:57
...,...,...,...,...,...,...,...
79,JFK,2022-02-13 23:51:00,0.0001,-SN,2022-02-13,2022-02-13 06:51:41,2022-02-13 17:27:29
80,JFK,2022-02-14 03:51:00,0.0001,-SN,2022-02-14,2022-02-14 06:50:25,2022-02-14 17:28:41
81,JFK,2022-02-25 06:51:00,1.5200,-FZRAPL,2022-02-25,2022-02-25 06:35:10,2022-02-25 17:41:41
82,JFK,2022-02-25 07:51:00,3.0500,PL BR,2022-02-25,2022-02-25 06:35:10,2022-02-25 17:41:41


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JFK,2022-12-12 02:51:00,1.0200,-RAPLSN BR,2022-12-12,2022-12-12 07:09:52,2022-12-12 16:27:50
1,JFK,2022-12-12 03:51:00,0.2500,-SN BR,2022-12-12,2022-12-12 07:09:52,2022-12-12 16:27:50
2,JFK,2022-12-12 04:51:00,0.2500,-SN BR,2022-12-12,2022-12-12 07:09:52,2022-12-12 16:27:50
3,JFK,2022-12-12 05:51:00,0.2500,-SN BR,2022-12-12,2022-12-12 07:09:52,2022-12-12 16:27:50
4,JFK,2022-12-12 06:51:00,0.5100,-SN BR,2022-12-12,2022-12-12 07:09:52,2022-12-12 16:27:50
5,JFK,2022-12-23 19:51:00,0.2500,-SN,2022-12-23,2022-12-23 07:16:40,2022-12-23 16:31:54
6,JFK,2023-01-14 09:51:00,0.0001,-SN,2023-01-14,2023-01-14 07:17:33,2023-01-14 16:51:02
7,JFK,2023-01-14 12:51:00,0.0001,-SN,2023-01-14,2023-01-14 07:17:33,2023-01-14 16:51:02
8,JFK,2023-01-14 13:51:00,0.0001,-SN,2023-01-14,2023-01-14 07:17:33,2023-01-14 16:51:02
9,JFK,2023-01-15 03:51:00,0.0001,-SN,2023-01-15,2023-01-15 07:17:10,2023-01-15 16:52:08


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JFK,2020-12-08 11:51:00,0.0000,-SN,2020-12-08,2020-12-08 07:06:58,2020-12-08 16:27:29
1,JFK,2020-12-08 12:51:00,0.0000,-SN,2020-12-08,2020-12-08 07:06:58,2020-12-08 16:27:29
16,JFK,2020-12-17 07:51:00,0.0000,-SNPL BR,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20
17,JFK,2020-12-17 08:51:00,0.0000,-SN BR BLSN,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20
18,JFK,2020-12-17 09:51:00,0.0000,-SN BLSN,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20
19,JFK,2020-12-17 10:51:00,0.0000,-SN BLSN,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20
20,JFK,2020-12-17 11:51:00,0.0000,-SN BLSN,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20
21,JFK,2020-12-17 12:51:00,0.0000,-SN BLSN,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20
22,JFK,2020-12-17 13:51:00,0.0000,-SN BLSN,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20
23,JFK,2020-12-17 14:51:00,0.0000,-SN BR,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
0,JFK,2020-12-08 11:51:00,0.0000,-SN,2020-12-08,2020-12-08 07:06:58,2020-12-08 16:27:29,2020-12-08 08:06:58,2020-12-08 15:27:29
1,JFK,2020-12-08 12:51:00,0.0000,-SN,2020-12-08,2020-12-08 07:06:58,2020-12-08 16:27:29,2020-12-08 08:06:58,2020-12-08 15:27:29
17,JFK,2020-12-17 08:51:00,0.0000,-SN BR BLSN,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20,2020-12-17 08:13:44,2020-12-17 15:29:20
18,JFK,2020-12-17 09:51:00,0.0000,-SN BLSN,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20,2020-12-17 08:13:44,2020-12-17 15:29:20
19,JFK,2020-12-17 10:51:00,0.0000,-SN BLSN,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20,2020-12-17 08:13:44,2020-12-17 15:29:20
20,JFK,2020-12-17 11:51:00,0.0000,-SN BLSN,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20,2020-12-17 08:13:44,2020-12-17 15:29:20
21,JFK,2020-12-17 12:51:00,0.0000,-SN BLSN,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20,2020-12-17 08:13:44,2020-12-17 15:29:20
22,JFK,2020-12-17 13:51:00,0.0000,-SN BLSN,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20,2020-12-17 08:13:44,2020-12-17 15:29:20
23,JFK,2020-12-17 14:51:00,0.0000,-SN BR,2020-12-17,2020-12-17 07:13:44,2020-12-17 16:29:20,2020-12-17 08:13:44,2020-12-17 15:29:20
29,JFK,2021-01-20 11:51:00,0.0001,-SN,2021-01-20,2021-01-20 07:14:25,2021-01-20 16:58:28,2021-01-20 08:14:25,2021-01-20 15:58:28


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.4407066666666667
2021-2022: 0.6191470588235295
2022-2023: 0.055633333333333326


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
2,JFK,2020-12-09 17:51:00,0.0000,-SN,2020-12-09,2020-12-09 07:07:50,2020-12-09 16:27:33
3,JFK,2020-12-09 18:51:00,0.0000,-SN BR,2020-12-09,2020-12-09 07:07:50,2020-12-09 16:27:33
4,JFK,2020-12-16 19:51:00,0.0001,-SN,2020-12-16,2020-12-16 07:13:05,2020-12-16 16:28:59
5,JFK,2020-12-16 20:51:00,0.2500,-SN,2020-12-16,2020-12-16 07:13:05,2020-12-16 16:28:59
6,JFK,2020-12-16 21:51:00,0.5100,-SN BR,2020-12-16,2020-12-16 07:13:05,2020-12-16 16:28:59
...,...,...,...,...,...,...,...
124,JFK,2021-02-19 21:51:00,0.0001,-SN BR,2021-02-19,2021-02-19 06:43:24,2021-02-19 17:34:57
125,JFK,2021-02-19 22:51:00,0.0001,-SN BR,2021-02-19,2021-02-19 06:43:24,2021-02-19 17:34:57
126,JFK,2021-02-19 23:51:00,0.2500,-SN BR,2021-02-19,2021-02-19 06:43:24,2021-02-19 17:34:57
127,JFK,2021-02-20 00:51:00,0.2500,-SN BR,2021-02-20,2021-02-20 06:42:01,2021-02-20 17:36:08


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_44148\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.4208368421052631
2021-2022: 0.016759999999999997
2022-2023: 0.15206


In [33]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('SO.jfk.csv', index=False)